# 📊 01 – Build Master Dataset for Market & Demographic Analysis

This notebook consolidates multiple datasets from the American Community Survey (ACS) 5-Year Estimates (2021) into a single master table. The goal is to prepare a unified dataset for exploratory analysis in support of the Ahsas School Market and Audience Project.

---

## 🔍 Data Sources

All data were downloaded from [https://data.census.gov](https://data.census.gov) and represent the most recent ACS 5-Year Estimates available at the **county subdivision** (town-level) geography for **Hartford County, Connecticut**.

The selected towns are:

- Bloomfield
- Windsor
- Windsor Locks
- West Hartford
- Hartford
- Simsbury
- Avon
- South Windsor
- Granby
- East Granby
- Suffield

---

## 📦 Included Datasets

| Table ID   | Description |
|------------|-------------|
| **S0101**  | Age and sex summary (broad age bands by gender)  
| **S1501**  | Educational attainment (total population age 25+)  
| **S1901**  | Household income (median, mean, and income brackets)  
| **B02001** | Race (population by single race category)  
| **B03002** | Hispanic or Latino origin by race  
| **C15002A–I** | Educational attainment by race and gender:  
→ A = White, B = Black, C = American Indian, D = Asian, E = Pacific Islander, F = Some Other Race, G = Two or More Races, H = White (non-Hispanic), I = Hispanic or Latino  
| **S1101**  | Households and families (e.g. average size, % with children under 6 and under 18, family type)  
| **S2301**  | Employment status (e.g. employed, unemployed, labor force participation)  
| **S2403**  | Industry of employment by gender (e.g. education, healthcare, professional services)

---

## 🎯 Objective

To merge all datasets into a single unified DataFrame `df_master` where:

- Each row = one town
- Each column = a demographic, educational, racial, income, household, or employment-related indicator

This master dataset will serve as the foundation for:

- Local market sizing
- Persona construction
- Segmentation and targeting
- School planning insights
- Competitive positioning analysis


In [1]:
# 📦 Basic Imports
import pandas as pd
import numpy as np
import os
import re

# 🖼️ Display Settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)  # Optional: set wider display

In [2]:
# not sharing the actual datasets here but tables can be visualized below and the end result is a clean dataset shared in the next notebook

base_path = "/DATA" 
print(os.listdir(base_path))


['productDownload_2025-06-05T204135', 'productDownload_2025-06-05T202238', 'ACSST5Y2021.S1501_2025-06-05T210821', 'ACSST5Y2021.S2403_2025-06-06T064745', 'ACSST5Y2021.S1101_2025-06-06T064129', '.DS_Store', 'productDownload_2025-06-05T202710', 'ACSST5Y2021.S2301_2025-06-06T064553', 'ACSST5Y2021.S0101_2025-06-05T201955', 'productDownload_2025-06-05T195552', 'clean_datasets', 'ACSDT5Y2021.B03002_2025-06-05T203555', 'ACSDT5Y2021.B02001_2025-06-05T202958', 'ACSST5Y2021.S1901_2025-06-05T210538', 'productDownload_2025-06-05T202707']


In [3]:
# 📂 Define base path
base_path = "/DATA"

# 📄 Updated file mapping with correct folder structure
file_map = {
    "s0101"   : "ACSST5Y2021.S0101_2025-06-05T201955/ACSST5Y2021.S0101-Data.csv",
    "s1501"   : "ACSST5Y2021.S1501_2025-06-05T210821/ACSST5Y2021.S1501-Data.csv",
    "s1901"   : "ACSST5Y2021.S1901_2025-06-05T210538/ACSST5Y2021.S1901-Data.csv",
    "b02001"  : "ACSDT5Y2021.B02001_2025-06-05T202958/ACSDT5Y2021.B02001-Data.csv",
    "b03002"  : "ACSDT5Y2021.B03002_2025-06-05T203555/ACSDT5Y2021.B03002-Data.csv",
    "s1101"  : "ACSST5Y2021.S1101_2025-06-06T064129/ACSST5Y2021.S1101-Data.csv",
    "s2301"  : "ACSST5Y2021.S2301_2025-06-06T064553/ACSST5Y2021.S2301-Data.csv",
    "s2403"  : "ACSST5Y2021.S2403_2025-06-06T064745/ACSST5Y2021.S2403-Data.csv",
    # All race-specific education tables (C15002A to I)
    "c15002a" : "productDownload_2025-06-05T195552/ACSDT5Y2021.C15002A-Data.csv",
    "c15002b" : "productDownload_2025-06-05T195552/ACSDT5Y2021.C15002B-Data.csv",
    "c15002c" : "productDownload_2025-06-05T195552/ACSDT5Y2021.C15002C-Data.csv",
    "c15002d" : "productDownload_2025-06-05T195552/ACSDT5Y2021.C15002D-Data.csv",
    "c15002e" : "productDownload_2025-06-05T195552/ACSDT5Y2021.C15002E-Data.csv",
    "c15002f" : "productDownload_2025-06-05T195552/ACSDT5Y2021.C15002F-Data.csv",
    "c15002g" : "productDownload_2025-06-05T195552/ACSDT5Y2021.C15002G-Data.csv",
    "c15002h" : "productDownload_2025-06-05T195552/ACSDT5Y2021.C15002H-Data.csv",
    "c15002i" : "productDownload_2025-06-05T195552/ACSDT5Y2021.C15002I-Data.csv",
}


# 📥 Load CSVs into a dictionary of DataFrames
dfs = {}
for key, relative_path in file_map.items():
    full_path = os.path.join(base_path, relative_path)
    try:
        df = pd.read_csv(full_path, skiprows=1)
        dfs[key] = df
        print(f"✅ Loaded: {key} → {df.shape[0]} rows, {df.shape[1]} columns")
    except FileNotFoundError:
        print(f"❌ File not found: {full_path}")

✅ Loaded: s0101 → 11 rows, 459 columns
✅ Loaded: s1501 → 11 rows, 771 columns
✅ Loaded: s1901 → 11 rows, 131 columns
✅ Loaded: b02001 → 11 rows, 23 columns
✅ Loaded: b03002 → 11 rows, 45 columns
✅ Loaded: s1101 → 11 rows, 193 columns
✅ Loaded: s2301 → 11 rows, 283 columns
✅ Loaded: s2403 → 11 rows, 273 columns
✅ Loaded: c15002a → 12 rows, 25 columns
✅ Loaded: c15002b → 12 rows, 25 columns
✅ Loaded: c15002c → 12 rows, 25 columns
✅ Loaded: c15002d → 12 rows, 25 columns
✅ Loaded: c15002e → 12 rows, 25 columns
✅ Loaded: c15002f → 12 rows, 25 columns
✅ Loaded: c15002g → 12 rows, 25 columns
✅ Loaded: c15002h → 12 rows, 25 columns
✅ Loaded: c15002i → 12 rows, 25 columns


In [4]:
# c15002 series has a column not-needed, Hartford County, so now we filter
# ✅ List of your 11 target towns
target_towns = [
    "Avon town, Hartford County, Connecticut",
    "Bloomfield town, Hartford County, Connecticut",
    "East Granby town, Hartford County, Connecticut",
    "Granby town, Hartford County, Connecticut",
    "Hartford town, Hartford County, Connecticut",
    "Simsbury town, Hartford County, Connecticut",
    "South Windsor town, Hartford County, Connecticut",
    "Suffield town, Hartford County, Connecticut",
    "West Hartford town, Hartford County, Connecticut",
    "Windsor town, Hartford County, Connecticut",
    "Windsor Locks town, Hartford County, Connecticut"
]

# 🧹 Filter and reset index for all c15002* DataFrames
for key in [k for k in dfs.keys() if k.startswith("c15002")]:
    df = dfs[key]
    df = df[df["Geographic Area Name"].isin(target_towns)].reset_index(drop=True)
    dfs[key] = df
    print(f"✅ Cleaned: {key} → {df.shape[0]} rows")

✅ Cleaned: c15002a → 11 rows
✅ Cleaned: c15002b → 11 rows
✅ Cleaned: c15002c → 11 rows
✅ Cleaned: c15002d → 11 rows
✅ Cleaned: c15002e → 11 rows
✅ Cleaned: c15002f → 11 rows
✅ Cleaned: c15002g → 11 rows
✅ Cleaned: c15002h → 11 rows
✅ Cleaned: c15002i → 11 rows


## 🧩 Data Preparation and Merging Workflow

This section focuses on preparing and consolidating the raw ACS datasets into a single unified DataFrame (`df_master`) for further analysis.

---

### 🔹 Step 1 – Select and Rename Relevant Columns

Each dataset contains numerous columns, including margins of error and verbose naming. We:

- Select only the **relevant estimate columns**
- Drop margins of error (`MOE`) and unnecessary metadata
- Rename columns for clarity and consistency (e.g., `Estimate!!Total:!!Male:!!5 to 9 years` → `male_5_9`)
- Standardize age, gender, education, and income group names

---

### 🔹 Step 2 – Standardize Town Identifiers

To enable accurate merging across datasets:

- We rename the geographic column to a consistent field: `town`
- Strip redundant text like `"town, Hartford County, Connecticut"` to extract clean town names
- Validate that all 11 selected towns match across all datasets

---

### 🔹 Step 3 – Merge All Datasets into `df_master`

We perform a sequence of **left joins** using `town` as the key. The result is a single, town-level dataset where:

- Each row = one of the 11 target towns
- Each column = a demographic, household, income, education, or employment indicator

This dataset is saved locally for reuse in future analysis phases.

```python
# Save master dataset to disk
df_master.to_csv("ahsas_master_dataset.csv", index=False)


In [5]:
s0101 = pd.read_csv(full_path, skiprows=1)
s0101.columns.tolist()

['Geography',
 'Geographic Area Name',
 'Estimate!!Total:',
 'Margin of Error!!Total:',
 'Estimate!!Total:!!Male:',
 'Margin of Error!!Total:!!Male:',
 'Estimate!!Total:!!Male:!!Less than high school diploma',
 'Margin of Error!!Total:!!Male:!!Less than high school diploma',
 'Estimate!!Total:!!Male:!!High school graduate (includes equivalency)',
 'Margin of Error!!Total:!!Male:!!High school graduate (includes equivalency)',
 "Estimate!!Total:!!Male:!!Some college or associate's degree",
 "Margin of Error!!Total:!!Male:!!Some college or associate's degree",
 "Estimate!!Total:!!Male:!!Bachelor's degree or higher",
 "Margin of Error!!Total:!!Male:!!Bachelor's degree or higher",
 'Estimate!!Total:!!Female:',
 'Margin of Error!!Total:!!Female:',
 'Estimate!!Total:!!Female:!!Less than high school diploma',
 'Margin of Error!!Total:!!Female:!!Less than high school diploma',
 'Estimate!!Total:!!Female:!!High school graduate (includes equivalency)',
 'Margin of Error!!Total:!!Female:!!High sch

In [6]:
# 📥 Load file
base_path = "/DATA"
s0101_path = os.path.join(base_path, "ACSST5Y2021.S0101_2025-06-05T201955", "ACSST5Y2021.S0101-Data.csv")
s0101 = pd.read_csv(s0101_path, skiprows=1)

# 🧽 Cleaning function
def clean_s0101(df):
    # Drop Margin of Error and unnamed columns
    df = df.loc[:, ~df.columns.str.contains("Margin of Error|Unnamed", case=False)].copy()

    # Rename "Geographic Area Name" to "town"
    df.rename(columns={"Geographic Area Name": "town"}, inplace=True)

    # Normalize and clean column names
    cleaned_cols = {}
    for col in df.columns:
        if col in ["Geography", "town"]:
            cleaned_cols[col] = col.lower()
        elif col.startswith("Estimate"):
            cleaned = col.replace("Estimate!!", "")
            cleaned = cleaned.replace("Total population", "total_population")
            cleaned = re.sub(r"!!", "_", cleaned)
            cleaned = cleaned.replace(" ", "_").lower()
            cleaned = re.sub(r"[()]", "", cleaned)
            cleaned = re.sub(r"_+", "_", cleaned).strip("_")
            cleaned_cols[col] = cleaned
        else:
            cleaned_cols[col] = col.lower()

    df.rename(columns=cleaned_cols, inplace=True)

    # 🔻 Drop unwanted patterns except some allowed (like median age)
    drop_patterns = [
        "percent_allocated",
        "dependency_ratio",
        "sex_ratio",
        "percent"
    ]
    keep_keywords = ["median_age_(years)"]

    columns_to_drop = [
        col for col in df.columns
        if any(p in col for p in drop_patterns)
        and not any(k in col for k in keep_keywords)
    ]
    df.drop(columns=columns_to_drop, inplace=True)

    # 🔢 Convert dashes and (X) to NaN, and numeric columns to float
    df.replace(["-", "(X)", "null"], np.nan, inplace=True)
    for col in df.columns:
        if col not in ["geography", "town"]:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # 🛑 DO NOT prefix here — will be done later via prepare_for_merge
    return df

# ✅ Apply cleaning
cleaned_s0101 = clean_s0101(s0101)

# 💾 Save if needed
cleaned_s0101.to_csv("/cleaned_s0101.csv", index=False)

# ✅ Confirm
print(f"✅ Cleaned s0101 shape: {cleaned_s0101.shape}")
print(cleaned_s0101.columns.tolist())

✅ Cleaned s0101 shape: (11, 98)
['geography', 'town', 'total_total_population', 'total_total_population_age_under_5_years', 'total_total_population_age_5_to_9_years', 'total_total_population_age_10_to_14_years', 'total_total_population_age_15_to_19_years', 'total_total_population_age_20_to_24_years', 'total_total_population_age_25_to_29_years', 'total_total_population_age_30_to_34_years', 'total_total_population_age_35_to_39_years', 'total_total_population_age_40_to_44_years', 'total_total_population_age_45_to_49_years', 'total_total_population_age_50_to_54_years', 'total_total_population_age_55_to_59_years', 'total_total_population_age_60_to_64_years', 'total_total_population_age_65_to_69_years', 'total_total_population_age_70_to_74_years', 'total_total_population_age_75_to_79_years', 'total_total_population_age_80_to_84_years', 'total_total_population_age_85_years_and_over', 'total_total_population_selected_age_categories_5_to_14_years', 'total_total_population_selected_age_categorie

## 🧹 Cleaning Summary – `s0101` (Age and Sex)

**Source File**: `ACSST5Y2021.S0101-Data.csv`  
**Location**: `/DATA/ACSST5Y2021.S0101_2025-06-05T201955/`

### 🔧 Cleaning Steps
1. **Loaded** the dataset, skipping metadata rows.
2. **Filtered** to retain only the 11 target towns in Hartford County.
3. **Dropped**:
   - All Margin of Error (MOE) columns.
   - All `Percent Allocated` columns.
   - All `Sex Ratio` and age dependency ratio indicators.
   - All percentage-based columns (percent_male, percent_female, etc.).
4. **Standardized column names**:
   - Replaced spaces and special characters with underscores.
   - Lowercased all names for consistency.
5. **Preserved**:
   - Total, male, and female population **counts** by age group.
   - Key age category aggregates (e.g., under 18, 65+).
   - **Median Age** for total, male, and female groups.

### 💾 Output
- **Saved File**: `s0101_cleaned.csv`
- **Location**: `/DATA/clean_datasets/`

In [7]:
def initial_clean_s1501(df):
    # Only keep 'Geography', 'Geographic Area Name', and Estimate!! columns
    cols_to_keep = ['Geography', 'Geographic Area Name'] + [
        col for col in df.columns if col.startswith("Estimate!!")
    ]
    df = df[cols_to_keep].copy()

    # Standardize column names
    def clean_col(col):
        if col in ['Geography', 'Geographic Area Name']:
            return col.lower().replace(" ", "_")
        col = col.replace("Estimate!!", "")
        col = col.replace("!!", "_")
        col = col.replace(" ", "_")
        col = col.replace(",", "")
        col = col.replace("(", "")
        col = col.replace(")", "")
        col = col.replace("-", "_")
        col = col.replace("'", "")
        return col.lower()

    df.columns = [clean_col(c) for c in df.columns]
    return df

In [8]:
def final_filter_s1501(df):
    return df[[c for c in df.columns if not (
        c.startswith("percent_") or c.startswith("male_") or c.startswith("female_")
    )]]

In [9]:
# Step-by-step application
dfs["s1501"] = initial_clean_s1501(dfs["s1501"])
dfs["s1501"] = final_filter_s1501(dfs["s1501"])

print(f"✅ Final s1501 shape: {dfs['s1501'].shape}")
dfs["s1501"].head()

✅ Final s1501 shape: (11, 66)


,geography,geographic_area_name,total_age_by_educational_attainment_population_18_to_24_years,total_age_by_educational_attainment_population_18_to_24_years_less_than_high_school_graduate,total_age_by_educational_attainment_population_18_to_24_years_high_school_graduate_includes_equivalency,total_age_by_educational_attainment_population_18_to_24_years_some_college_or_associates_degree,total_age_by_educational_attainment_population_18_to_24_years_bachelors_degree_or_higher,total_age_by_educational_attainment_population_25_years_and_over,total_age_by_educational_attainment_population_25_years_and_over_less_than_9th_grade,total_age_by_educational_attainment_population_25_years_and_over_9th_to_12th_grade_no_diploma,total_age_by_educational_attainment_population_25_years_and_over_high_school_graduate_includes_equivalency,total_age_by_educational_attainment_population_25_years_and_over_some_college_no_degree,total_age_by_educational_attainment_population_25_years_and_over_associates_degree,total_age_by_educational_attainment_population_25_years_and_over_bachelors_degree,total_age_by_educational_attainment_population_25_years_and_over_graduate_or_professional_degree,total_age_by_educational_attainment_population_25_years_and_over_high_school_graduate_or_higher,total_age_by_educational_attainment_population_25_years_and_over_bachelors_degree_or_higher,total_age_by_educational_attainment_population_25_to_34_years,total_age_by_educational_attainment_population_25_to_34_years_high_school_graduate_or_higher,total_age_by_educational_attainment_population_25_to_34_years_bachelors_degree_or_higher,total_age_by_educational_attainment_population_35_to_44_years,total_age_by_educational_attainment_population_35_to_44_years_high_school_graduate_or_higher,total_age_by_educational_attainment_population_35_to_44_years_bachelors_degree_or_higher,total_age_by_educational_attainment_population_45_to_64_years,total_age_by_educational_attainment_population_45_to_64_years_high_school_graduate_or_higher,total_age_by_educational_attainment_population_45_to_64_years_bachelors_degree_or_higher,total_age_by_educational_attainment_population_65_years_and_over,total_age_by_educational_attainment_population_65_years_and_over_high_school_graduate_or_higher,total_age_by_educational_attainment_population_65_years_and_over_bachelors_degree_or_higher,total_race_and_hispanic_or_latino_origin_by_educational_attainment_white_alone,total_race_and_hispanic_or_latino_origin_by_educational_attainment_white_alone_high_school_graduate_or_higher,total_race_and_hispanic_or_latino_origin_by_educational_attainment_white_alone_bachelors_degree_or_higher,total_race_and_hispanic_or_latino_origin_by_educational_attainment_white_alone_not_hispanic_or_latino,total_race_and_hispanic_or_latino_origin_by_educational_attainment_white_alone_not_hispanic_or_latino_high_school_graduate_or_higher,total_race_and_hispanic_or_latino_origin_by_educational_attainment_white_alone_not_hispanic_or_latino_bachelors_degree_or_higher,total_race_and_hispanic_or_latino_origin_by_educational_attainment_black_alone,total_race_and_hispanic_or_latino_origin_by_educational_attainment_black_alone_high_school_graduate_or_higher,total_race_and_hispanic_or_latino_origin_by_educational_attainment_black_alone_bachelors_degree_or_higher,total_race_and_hispanic_or_latino_origin_by_educational_attainment_american_indian_or_alaska_native_alone,total_race_and_hispanic_or_latino_origin_by_educational_attainment_american_indian_or_alaska_native_alone_high_school_graduate_or_higher,total_race_and_hispanic_or_latino_origin_by_educational_attainment_american_indian_or_alaska_native_alone_bachelors_degree_or_higher,total_race_and_hispanic_or_latino_origin_by_educational_attainment_asian_alone,total_race_and_hispanic_or_latino_origin_by_educational_attainment_asian_alone_high_school_graduate_or_higher,total_race_and_hispanic_or_latino_origin_by_educational_attainment_asian_alone_bachelors_degree_or_higher,total_race_and_hispanic_o

In [10]:
s1501_cols = dfs["s1501"].columns.tolist()
for col in s1501_cols:
    print(col)

geography
geographic_area_name
total_age_by_educational_attainment_population_18_to_24_years
total_age_by_educational_attainment_population_18_to_24_years_less_than_high_school_graduate
total_age_by_educational_attainment_population_18_to_24_years_high_school_graduate_includes_equivalency
total_age_by_educational_attainment_population_18_to_24_years_some_college_or_associates_degree
total_age_by_educational_attainment_population_18_to_24_years_bachelors_degree_or_higher
total_age_by_educational_attainment_population_25_years_and_over
total_age_by_educational_attainment_population_25_years_and_over_less_than_9th_grade
total_age_by_educational_attainment_population_25_years_and_over_9th_to_12th_grade_no_diploma
total_age_by_educational_attainment_population_25_years_and_over_high_school_graduate_includes_equivalency
total_age_by_educational_attainment_population_25_years_and_over_some_college_no_degree
total_age_by_educational_attainment_population_25_years_and_over_associates_degree
tot

In [11]:
# Define output path
output_path = "/cleaned_s1501.csv"

# Save the DataFrame
dfs["s1501"].to_csv(output_path, index=False)
print(f"✅ Saved: {output_path}")

✅ Saved: /Users/marcomagnolo/Desktop/NEXT/pro_bono_projects/ahsas_school/DATA/clean_datasets/cleaned_s1501.csv


# 🧼 Cleaning Summary: `s1501` – Educational Attainment Dataset

**Source**: `ACSST5Y2021.S1501-Data.csv`  
**Location**: `/DATA/ACSST5Y2021.S1501_2025-06-05T210821/`  
**Output**: `cleaned_s1501.csv` in `/DATA/clean_datasets/`

---

## ✅ Cleaning Steps

1. **Column Filtering**
   - Kept:
     - `Geography`, `Geographic Area Name`
     - Columns starting with `Estimate!!` (raw counts only)
   - Removed:
     - All `Margin of Error` and `Percent` columns
     - All gender-specific estimates: `Male!!`, `Female!!`

   > ⚠️ *Note: Gender-specific columns were removed due to high sparsity and limited relevance to current project goals. This decision is flagged for future review in case gender-based segmentation becomes a priority.*

2. **Column Name Standardization**
   - Converted to `snake_case`
   - Removed special characters (`!!`, commas, dashes, parentheses, etc.)

3. **Final Filtering**
   - Dropped columns beginning with:
     - `percent_`
     - `male_`
     - `female_`

---

## 📊 Final Result

- **Rows**: 11 (one per target town)
- **Columns**: 66
- **Key Themes**:
  - Educational attainment by age group
  - Educational attainment by race/ethnicity
  - Poverty by education level
  - Median earnings by education level

---

✔️ Ready for integration into the master dataset.


In [12]:
def clean_s1901(df):
    # Step 1: Keep only 'Geography', 'Geographic Area Name', and Estimate!! columns
    cols_to_keep = ['Geography', 'Geographic Area Name'] + [
        col for col in df.columns if col.startswith("Estimate!!")
    ]
    df = df[cols_to_keep].copy()

    # Step 2: Drop low-utility columns (e.g., 'PERCENT ALLOCATED')
    drop_keywords = ["PERCENT ALLOCATED"]
    df = df[[col for col in df.columns if not any(kw in col for kw in drop_keywords)]]

    # Step 3: Standardize column names
    def clean_col(col):
        if col in ['Geography', 'Geographic Area Name']:
            return col.lower().replace(" ", "_")
        col = col.replace("Estimate!!", "")
        col = col.replace("!!", "_")
        col = col.replace(" ", "_")
        col = col.replace(",", "")
        col = col.replace("(", "")
        col = col.replace(")", "")
        col = col.replace("-", "_")
        col = col.replace("'", "")
        return col.lower()

    df.columns = [clean_col(c) for c in df.columns]
    
    return df

In [13]:
# Apply the cleaning function
dfs["s1901"] = clean_s1901(dfs["s1901"])
print(f"✅ Cleaned s1901 shape: {dfs['s1901'].shape}")
dfs["s1901"].head()

✅ Cleaned s1901 shape: (11, 54)


,geography,geographic_area_name,households_total,households_total_less_than_$10000,households_total_$10000_to_$14999,households_total_$15000_to_$24999,households_total_$25000_to_$34999,households_total_$35000_to_$49999,households_total_$50000_to_$74999,households_total_$75000_to_$99999,households_total_$100000_to_$149999,households_total_$150000_to_$199999,households_total_$200000_or_more,households_median_income_dollars,households_mean_income_dollars,families_total,families_total_less_than_$10000,families_total_$10000_to_$14999,families_total_$15000_to_$24999,families_total_$25000_to_$34999,families_total_$35000_to_$49999,families_total_$50000_to_$74999,families_total_$75000_to_$99999,families_total_$100000_to_$149999,families_total_$150000_to_$199999,families_total_$200000_or_more,families_median_income_dollars,families_mean_income_dollars,married_couple_families_total,married_couple_families_total_less_than_$10000,married_couple_families_total_$10000_to_$14999,married_couple_families_total_$15000_to_$24999,married_couple_families_total_$25000_to_$34999,married_couple_families_total_$35000_to_$49999,married_couple_families_total_$50000_to_$74999,married_couple_families_total_$75000_to_$99999,married_couple_families_total_$100000_to_$149999,married_couple_families_total_$150000_to_$199999,married_couple_families_total_$200000_or_more,married_couple_families_median_income_dollars,married_couple_families_mean_income_dollars,nonfamily_households_total,nonfamily_households_total_less_than_$10000,nonfamily_households_total_$10000_to_$14999,nonfamily_households_total_$15000_to_$24999,nonfamily_households_total_$25000_to_$34999,nonfamily_households_total_$35000_to_$49999,nonfamily_households_total_$50000_to_$74999,nonfamily_households_total_$75000_to_$99999,nonfamily_households_total_$100000_to_$149999,nonfamily_households_total_$150000_to_$199999,nonfamily_households_total_$200000_or_more,nonfamily_households_median_income_dollars,nonfamily_households_mean_income_dollars
0,0600000US0900302060,"Avon town, Hartford County, Connecticut",7398,1.4,1.8,4.1,7.0,8.6,7.5,8.4,17.3,13.3,30.6,130268,201572,5259,0.1,1.7,2.3,3.0,3.5,6.1,9.2,18.1,17.8,38.2,166926,247815,4761,0.1,1.6,1.3,2.2,3.0,6.1,8.3,18.4,17.6,41.4,173876,N,2139,4.4,1.8,8.8,16.7,21.3,12.4,7.3,15.9,2.0,9.3,46010,82315
1,0600000US0900305910,"Bloomfield town, Hartford County, Connecticut",8802,4.4,2.8,4.9,7.6,10.9,15.1,14.9,19.5,10.6,9.3,81354,103070,5385,2.0,2.7,1.8,4.0,9.9,15.0,15.7,22.7,14.2,12.0,97917,118721,3596,0.3,0.6,2.6,2.3,7.2,13.6,14.8,25.8,16.7,16.0,110423,N,3417,8.1,4.4,10.0,14.0,12.6,16.9,13.2,13.2,3.5,4.1,51125,73182
2,0600000US0900322070,"East Granby town, Hartford County, Connecticut",1896,1.1,2.0,4.8,5.2,8.9,15.6,10.1,19.3,15.1,18.0,112857,119605,1317,0.0,0.0,0.0,0.0,7.4,13.4,19.0,18.1,19.9,22.1,127708,139830,988,0.0,0.0,0.0,0.0,3.4,11.6,11.4,22.8,24.8,25.9,150565,N,579,3.6,6.6,15.7,16.9,12.1,20.4,4.5,14.3,1.9,4.0,43029,58745
3,0600000US0900332640,"Granby town, Hartford County, Connecticut",4247,3.5,0.6,1.3,4.0,12.0,12.8,9.8,18.7,15.9,21.4,115989,147448,3151,0.5,0.3,0.6,1.9,7.1,13.5,9.7,22.3,17.9,26.3,139149,169310,2842,0.0,0.4,0.4,2.1,6.7,11.2,8.8,22.8,18.8,28.8,143512,N,1096,12.3,1.4,3.4,10.1,27.6,13.6,8.3,11.9,4.3,7.2,44474,81036
4,0600000US0900337070,"Hartford town, Hartford County, Connecticut",46879,14.0,9.6,11.3,13.5,12.0,15.3,8.6,9.1,4.3,2.4,37477,56597,26099,9.6,5.5,10.9,13.3,15.9,16.1,9.6,11.3,4.7,3.0,44859,64996,10269,4.2,2.6,5.1,9.9,13.7,18.1,15.5,15.2,9.0,6.7,69315,93614,20780,20.7,15.4,13.2,14.8,8.6,11.5,6.1,5.3,3.3,1.2,25401,41283


In [14]:
s1901_cols = dfs["s1901"].columns.tolist()
for col in s1901_cols:
    print(col)

geography
geographic_area_name
households_total
households_total_less_than_$10000
households_total_$10000_to_$14999
households_total_$15000_to_$24999
households_total_$25000_to_$34999
households_total_$35000_to_$49999
households_total_$50000_to_$74999
households_total_$75000_to_$99999
households_total_$100000_to_$149999
households_total_$150000_to_$199999
households_total_$200000_or_more
households_median_income_dollars
households_mean_income_dollars
families_total
families_total_less_than_$10000
families_total_$10000_to_$14999
families_total_$15000_to_$24999
families_total_$25000_to_$34999
families_total_$35000_to_$49999
families_total_$50000_to_$74999
families_total_$75000_to_$99999
families_total_$100000_to_$149999
families_total_$150000_to_$199999
families_total_$200000_or_more
families_median_income_dollars
families_mean_income_dollars
married_couple_families_total
married_couple_families_total_less_than_$10000
married_couple_families_total_$10000_to_$14999
married_couple_families

In [15]:
# Define output path
output_path = "/cleaned_s1901.csv"

# Save the cleaned DataFrame
dfs["s1901"].to_csv(output_path, index=False)
print(f"✅ Saved: {output_path}")

✅ Saved: /Users/marcomagnolo/Desktop/NEXT/pro_bono_projects/ahsas_school/DATA/clean_datasets/cleaned_s1901.csv


# 🧼 Cleaning Summary: `s1901` – Income in the Past 12 Months

**Source**: `ACSST5Y2021.S1901-Data.csv`  
**Location**: `/DATA/ACSST5Y2021.S1901_2025-06-05T210538/`  
**Output**: `cleaned_s1901.csv` in `/DATA/clean_datasets/`

---

## ✅ Cleaning Steps

1. **Column Filtering**
   - Kept:
     - `Geography`, `Geographic Area Name`
     - All columns starting with `Estimate!!`
   - Removed:
     - All `Margin of Error!!` columns
     - All `PERCENT ALLOCATED` columns (metadata for imputed values)

2. **Column Name Standardization**
   - Converted to `snake_case`
   - Removed special characters (`!!`, commas, dashes, parentheses, etc.)

---

## 📊 Final Result

- **Rows**: 11 (one per target town)
- **Columns**: 54
- **Key Themes**:
  - Income brackets for households, families, married-couple families, and nonfamily households
  - Median and mean income for each category

---

## 🧠 Notes

- `households_total` includes both `families_total` and `nonfamily_households_total`
- `married_couple_families_total` is a **subset** of `families_total`
- Remaining family types (e.g., single-parent households) are captured in the difference between `families_total` and `married_couple_families_total`

> 🔍 **About the "Past 12 Months" Definition**  
> All income figures in `s1901` reflect income earned in the 12 months prior to each respondent’s interview date. Since this table is based on the **2017–2021 ACS 5-Year Estimates**, the data collection spans across **five full years**, covering diverse economic conditions.  
>  
> This rolling design provides **greater statistical stability** and smooths out short-term economic fluctuations (e.g., temporary job loss or pandemic effects).  
>  
> All income values are **inflation-adjusted to 2021 dollars**, making the table suitable for reliable cross-community comparisons in planning and strategy.

---

✔️ Cleaned and ready for integration into the master dataset.


In [16]:
def clean_b02001(df):
    # Step 1: Keep only 'Geography', 'Geographic Area Name', and Estimate columns
    cols_to_keep = ['Geography', 'Geographic Area Name'] + [
        col for col in df.columns if col.startswith("Estimate!!")
    ]
    df = df[cols_to_keep].copy()

    # Step 2: Standardize column names
    def clean_col(col):
        if col in ['Geography', 'Geographic Area Name']:
            return col.lower().replace(" ", "_")
        col = col.replace("Estimate!!", "")
        col = col.replace("!!", "_")
        col = col.replace(" ", "_")
        col = col.replace(",", "")
        col = col.replace("(", "")
        col = col.replace(")", "")
        col = col.replace("-", "_")
        col = col.replace("'", "")
        return col.lower()

    df.columns = [clean_col(c) for c in df.columns]
    
    return df

In [17]:
# Apply the cleaner
dfs["b02001"] = clean_b02001(dfs["b02001"])
print(f"✅ Cleaned b02001 shape: {dfs['b02001'].shape}")
dfs["b02001"].head()

✅ Cleaned b02001 shape: (11, 12)


,geography,geographic_area_name,total:,total:_white_alone,total:_black_or_african_american_alone,total:_american_indian_and_alaska_native_alone,total:_asian_alone,total:_native_hawaiian_and_other_pacific_islander_alone,total:_some_other_race_alone,total:_two_or_more_races:,total:_two_or_more_races:_two_races_including_some_other_race,total:_two_or_more_races:_two_races_excluding_some_other_race_and_three_or_more_races
0,0600000US0900302060,"Avon town, Hartford County, Connecticut",18821,14879,208,165,2672,0,334,563,188,375
1,0600000US0900305910,"Bloomfield town, Hartford County, Connecticut",21399,7140,12194,0,898,0,316,851,306,545
2,0600000US0900322070,"East Granby town, Hartford County, Connecticut",5148,4068,276,0,163,0,82,559,85,474
3,0600000US0900332640,"Granby town, Hartford County, Connecticut",10971,10368,76,0,127,0,53,347,243,104
4,0600000US0900337070,"Hartford town, Hartford County, Connecticut",121562,33756,44259,630,3122,79,25189,14527,5257,9270


In [18]:
# Define output path
output_path = "/cleaned_b02001.csv"

# Save the cleaned DataFrame
dfs["b02001"].to_csv(output_path, index=False)
print(f"✅ Saved: {output_path}")

✅ Saved: /Users/marcomagnolo/Desktop/NEXT/pro_bono_projects/ahsas_school/DATA/clean_datasets/cleaned_b02001.csv


# 🧼 Cleaning Summary: `b02001` – Race Table

**Source**: `ACSDT5Y2021.B02001-Data.csv`  
**Location**: `/DATA/ACSDT5Y2021.B02001_2025-06-05T202958/`  
**Output**: `cleaned_b02001.csv` in `/DATA/clean_datasets/`

---

## ✅ Cleaning Steps

1. **Column Filtering**
   - Kept:
     - `Geography`, `Geographic Area Name`
     - All columns starting with `Estimate!!` (raw population counts)
   - Removed:
     - All columns starting with `Margin of Error!!`
     - Any empty or unnamed columns

2. **Column Name Standardization**
   - Converted to `snake_case`
   - Removed special characters (`!!`, commas, dashes, colons, etc.)

---

## 📊 Final Result

- **Rows**: 11 (one per target town)
- **Columns**: 12
- **Key Themes**:
  - Population totals by race group:
    - White alone
    - Black or African American alone
    - American Indian and Alaska Native alone
    - Asian alone
    - Native Hawaiian and Other Pacific Islander alone
    - Some other race alone
    - Two or more races

---

✔️ Cleaned and ready for demographic integration.

In [19]:
def clean_b03002(df):
    # Step 1: Keep only Geography and Estimate columns
    cols_to_keep = ['Geography', 'Geographic Area Name'] + [
        col for col in df.columns if col.startswith("Estimate!!")
    ]
    df = df[cols_to_keep].copy()

    # Step 2: Drop Native Hawaiian and Other Pacific Islander rows
    drop_substrings = [
        "Native Hawaiian and Other Pacific Islander alone",
        "Margin of Error!!"
    ]
    df = df[[col for col in df.columns if not any(kw in col for kw in drop_substrings)]]

    # Step 3: Standardize column names
    def clean_col(col):
        if col in ['Geography', 'Geographic Area Name']:
            return col.lower().replace(" ", "_")
        col = col.replace("Estimate!!", "")
        col = col.replace("!!", "_")
        col = col.replace(" ", "_")
        col = col.replace(",", "")
        col = col.replace("(", "")
        col = col.replace(")", "")
        col = col.replace("-", "_")
        col = col.replace("'", "")
        return col.lower()

    df.columns = [clean_col(c) for c in df.columns]

    return df

In [20]:
# Clean and store in dfs dictionary
dfs["b03002"] = clean_b03002(dfs["b03002"])
print(f"✅ Cleaned b03002 shape: {dfs['b03002'].shape}")
dfs["b03002"].head()

✅ Cleaned b03002 shape: (11, 21)


,geography,geographic_area_name,total:,total:_not_hispanic_or_latino:,total:_not_hispanic_or_latino:_white_alone,total:_not_hispanic_or_latino:_black_or_african_american_alone,total:_not_hispanic_or_latino:_american_indian_and_alaska_native_alone,total:_not_hispanic_or_latino:_asian_alone,total:_not_hispanic_or_latino:_some_other_race_alone,total:_not_hispanic_or_latino:_two_or_more_races:,total:_not_hispanic_or_latino:_two_or_more_races:_two_races_including_some_other_race,total:_not_hispanic_or_latino:_two_or_more_races:_two_races_excluding_some_other_race_and_three_or_more_races,total:_hispanic_or_latino:,total:_hispanic_or_latino:_white_alone,total:_hispanic_or_latino:_black_or_african_american_alone,total:_hispanic_or_latino:_american_indian_and_alaska_native_alone,total:_hispanic_or_latino:_asian_alone,total:_hispanic_or_latino:_some_other_race_alone,total:_hispanic_or_latino:_two_or_more_races:,total:_hispanic_or_latino:_two_or_more_races:_two_races_including_some_other_race,total:_hispanic_or_latino:_two_or_more_races:_two_races_excluding_some_other_race_and_three_or_more_races
0,0600000US0900302060,"Avon town, Hartford County, Connecticut",18821,18149,14622,173,7,2672,181,494,127,367,672,257,35,158,0,153,69,61,8
1,0600000US0900305910,"Bloomfield town, Hartford County, Connecticut",21399,19889,6461,11979,0,898,124,427,55,372,1510,679,215,0,0,192,424,251,173
2,0600000US0900322070,"East Granby town, Hartford County, Connecticut",5148,4794,3929,276,0,163,0,426,85,341,354,139,0,0,0,82,133,0,133
3,0600000US0900332640,"Granby town, Hartford County, Connecticut",10971,10698,10303,76,0,127,0,192,99,93,273,65,0,0,0,53,155,144,11
4,0600000US0900337070,"Hartford town, Hartford County, Connecticut",121562,66307,17843,41439,74,3111,746,3094,714,2380,55255,15913,2820,556,11,24443,11433,4543,6890


In [21]:
# Define output path
output_path = "cleaned_b03002.csv"

# Save the cleaned DataFrame
dfs["b03002"].to_csv(output_path, index=False)
print(f"✅ Saved: {output_path}")

✅ Saved: /Users/marcomagnolo/Desktop/NEXT/pro_bono_projects/ahsas_school/DATA/clean_datasets/cleaned_b03002.csv


# 🧼 Cleaning Summary: `b03002` – Hispanic or Latino Origin by Race

**Source**: `ACSDT5Y2021.B03002-Data.csv`  
**Location**: `/DATA/ACSDT5Y2021.B03002_2025-06-05T203555/`  
**Output**: `cleaned_b03002.csv` in `/DATA/clean_datasets/`

---

## ✅ Cleaning Steps

1. **Column Filtering**
   - Kept:
     - `Geography`, `Geographic Area Name`
     - All `Estimate!!` columns with relevant race and ethnicity breakdowns
   - Removed:
     - All `Margin of Error!!` columns
     - All race subgroups involving `Native Hawaiian and Other Pacific Islander alone` (too small to analyze in this context)

2. **Column Name Standardization**
   - Converted to `snake_case`
   - Removed special characters (`!!`, colons, commas, dashes, parentheses, etc.)

---

## 📊 Final Result

- **Rows**: 11 (one per target town)
- **Columns**: *[count will vary]* depending on final retained estimates
- **Key Themes**:
  - Total population
  - Hispanic or Latino population (any race)
  - Non-Hispanic population by race
  - Race/ethnicity intersections (e.g., White Non-Hispanic, Hispanic Some Other Race)

---

## 🧠 Notes

> `b03002` is the only ACS table that explicitly distinguishes **ethnicity** (Hispanic/Latino vs. Not) from **race**.  
> It complements `b02001`, allowing for culturally aware segmentation and accurate population counts that reflect both identity dimensions.  
>  
> Some subgroups like `Some other race alone` under Hispanic are often **very high**, especially in Latinx communities where U.S. racial categories don’t align with lived identity.

✔️ Cleaned and ready for demographic integration.

In [22]:
def refined_clean_s1101(df):
    # Step 1: Keep only Geography and Estimate columns
    cols_to_keep = ['Geography', 'Geographic Area Name'] + [
        col for col in df.columns if col.startswith("Estimate!!")
    ]
    df = df[cols_to_keep].copy()

    # Step 2: Clean column names
    def clean_col(col):
        if col in ['Geography', 'Geographic Area Name']:
            return col.lower().replace(" ", "_")
        col = col.replace("Estimate!!", "")
        col = col.replace("!!", "_")
        col = col.replace(" ", "_")
        col = col.replace(",", "")
        col = col.replace("(", "")
        col = col.replace(")", "")
        col = col.replace("-", "_")
        col = col.replace("'", "")
        return col.lower()

    df.columns = [clean_col(c) for c in df.columns]

    # Step 3: Drop low-utility subgroups and housing details
    drop_prefixes = [
        "married_couple_family_household_",
        "male_householder_no_spouse_present_family_household_",
        "female_householder_no_spouse_present_family_household_",
        "nonfamily_household_",
        "total_total_households_units_in_structure_",
        "total_total_households_housing_tenure_"
    ]

    df = df[[col for col in df.columns if not any(col.startswith(prefix) for prefix in drop_prefixes)]]

    return df

In [23]:
dfs["s1101"] = refined_clean_s1101(dfs["s1101"])
print(f"✅ Refined s1101 shape: {dfs['s1101'].shape}")
dfs["s1101"].head()

✅ Refined s1101 shape: (11, 16)


,geography,geographic_area_name,total_households_total_households,total_households_average_household_size,total_families_total_families,total_families_average_family_size,total_age_of_own_children_households_with_own_children_of_the_householder_under_18_years,total_age_of_own_children_households_with_own_children_of_the_householder_under_18_years_under_6_years_only,total_age_of_own_children_households_with_own_children_of_the_householder_under_18_years_under_6_years_and_6_to_17_years,total_age_of_own_children_households_with_own_children_of_the_householder_under_18_years_6_to_17_years_only,total_total_households,total_total_households_selected_households_by_type_households_with_one_or_more_people_under_18_years,total_total_households_selected_households_by_type_households_with_one_or_more_people_60_years_and_over,total_total_households_selected_households_by_type_households_with_one_or_more_people_65_year_and_over,total_total_households_selected_households_by_type_householder_living_alone,total_total_households_selected_households_by_type_householder_living_alone_65_years_and_over
0,0600000US0900302060,"Avon town, Hartford County, Connecticut",7398,2.51,5259,3.04,2461,15.9,12.6,71.6,7398,33.7,48.1,36.3,26.7,13.9
1,0600000US0900305910,"Bloomfield town, Hartford County, Connecticut",8802,2.37,5385,3.02,1604,13.8,20.4,65.8,8802,22.1,51.7,41.6,31.4,15.1
2,0600000US0900322070,"East Granby town, Hartford County, Connecticut",1896,2.72,1317,3.08,679,18.3,8.8,72.9,1896,39.5,40.1,35.1,23.3,11.8
3,0600000US0900332640,"Granby town, Hartford County, Connecticut",4247,2.56,3151,2.97,1273,30.0,13.4,56.6,4247,31.2,51.9,35.2,21.3,10.3
4,0600000US0900337070,"Hartford town, Hartford County, Connecticut",46879,2.43,26099,3.24,12863,19.8,22.1,58.0,46879,32.4,33.1,24.2,37.7,11.5


In [24]:
# Get column list
s1101_cols = dfs["s1101"].columns.tolist()

# Print them all
for col in s1101_cols:
    print(col)

geography
geographic_area_name
total_households_total_households
total_households_average_household_size
total_families_total_families
total_families_average_family_size
total_age_of_own_children_households_with_own_children_of_the_householder_under_18_years
total_age_of_own_children_households_with_own_children_of_the_householder_under_18_years_under_6_years_only
total_age_of_own_children_households_with_own_children_of_the_householder_under_18_years_under_6_years_and_6_to_17_years
total_age_of_own_children_households_with_own_children_of_the_householder_under_18_years_6_to_17_years_only
total_total_households
total_total_households_selected_households_by_type_households_with_one_or_more_people_under_18_years
total_total_households_selected_households_by_type_households_with_one_or_more_people_60_years_and_over
total_total_households_selected_households_by_type_households_with_one_or_more_people_65_year_and_over
total_total_households_selected_households_by_type_householder_living_alo

In [25]:
# Define output path
output_path = "cleaned_s1101.csv"

# Save the cleaned DataFrame
dfs["s1101"].to_csv(output_path, index=False)
print(f"✅ Saved: {output_path}")

✅ Saved: /Users/marcomagnolo/Desktop/NEXT/pro_bono_projects/ahsas_school/DATA/clean_datasets/cleaned_s1101.csv


# 🧼 Cleaning Summary: `s1101` – Households and Families

**Source**: `ACSST5Y2021.S1101-Data.csv`  
**Location**: `/DATA/ACSST5Y2021.S1101_2025-06-06T064129/`  
**Output**: `cleaned_s1101.csv` in `/DATA/clean_datasets/`

---

## ✅ Cleaning Steps

1. **Column Filtering**
   - Kept:
     - `Geography`, `Geographic Area Name`
     - All `Estimate!!` columns related to total households, families, child presence, elderly presence, and living alone
   - Removed:
     - All `Margin of Error!!` columns
     - All columns related to:
       - Household breakdowns by marital/gender status (e.g., married couples, male/female householders)
       - Housing structure type (e.g., 1-unit, mobile homes)
       - Housing tenure (e.g., owner vs. renter)

2. **Column Name Standardization**
   - Converted to `snake_case`
   - Removed special characters (`!!`, commas, dashes, parentheses, etc.)

---

## 📊 Final Result

- **Rows**: 11 (one per target town)
- **Columns**: 16
- **Key Themes**:
  - Total households and families
  - Household and family size
  - Presence and age of children under 18
  - Presence of older adults (60+, 65+)
  - Householders living alone, especially seniors

---

## 🧠 Notes

> `s1101` provides structural insights into the composition of households and families, helping identify populations with children, seniors, or multigenerational arrangements.  
> The refined version focuses on actionable segments relevant to school outreach, community engagement, and service planning.

✔️ Cleaned and ready for integration into the master dataset.

In [26]:
def clean_s2301(df):
    # Step 1: Keep only Geography and Estimate columns
    cols_to_keep = ['Geography', 'Geographic Area Name'] + [
        col for col in df.columns if col.startswith("Estimate!!")
    ]
    df = df[cols_to_keep].copy()

    # Step 2: Drop overly nested or demographic-specific breakdowns
    drop_keywords = [
        "!!SEX!!", "!!RACE!!", "!!HISPANIC OR LATINO OR NOT HISPANIC OR LATINO!!",
        "!!White alone", "!!Black or African American alone",
        "!!Asian alone", "!!Some other race alone", "!!Two or more races"
    ]
    df = df[[col for col in df.columns if not any(kw in col for kw in drop_keywords)]]

    # Step 3: Standardize column names
    def clean_col(col):
        if col in ['Geography', 'Geographic Area Name']:
            return col.lower().replace(" ", "_")
        col = col.replace("Estimate!!", "")
        col = col.replace("!!", "_")
        col = col.replace(" ", "_")
        col = col.replace(",", "")
        col = col.replace("(", "")
        col = col.replace(")", "")
        col = col.replace("-", "_")
        col = col.replace("'", "")
        return col.lower()

    df.columns = [clean_col(c) for c in df.columns]

    return df

In [27]:
dfs["s2301"] = clean_s2301(dfs["s2301"])
print(f"✅ Cleaned s2301 shape: {dfs['s2301'].shape}")
dfs["s2301"].head()

✅ Cleaned s2301 shape: (11, 94)


,geography,geographic_area_name,total_population_16_years_and_over,total_population_16_years_and_over_age_16_to_19_years,total_population_16_years_and_over_age_20_to_24_years,total_population_16_years_and_over_age_25_to_29_years,total_population_16_years_and_over_age_30_to_34_years,total_population_16_years_and_over_age_35_to_44_years,total_population_16_years_and_over_age_45_to_54_years,total_population_16_years_and_over_age_55_to_59_years,total_population_16_years_and_over_age_60_to_64_years,total_population_16_years_and_over_age_65_to_74_years,total_population_16_years_and_over_age_75_years_and_over,total_population_16_years_and_over_race_and_hispanic_or_latino_origin_american_indian_and_alaska_native_alone,total_population_16_years_and_over_race_and_hispanic_or_latino_origin_native_hawaiian_and_other_pacific_islander_alone,total_population_16_years_and_over_race_and_hispanic_or_latino_origin_hispanic_or_latino_origin_of_any_race,total_population_20_to_64_years,total_population_20_to_64_years_poverty_status_in_the_past_12_months_below_poverty_level,total_population_20_to_64_years_poverty_status_in_the_past_12_months_at_or_above_the_poverty_level,total_population_20_to_64_years_disability_status_with_any_disability,total_educational_attainment_population_25_to_64_years,total_educational_attainment_population_25_to_64_years_less_than_high_school_graduate,total_educational_attainment_population_25_to_64_years_high_school_graduate_includes_equivalency,total_educational_attainment_population_25_to_64_years_some_college_or_associates_degree,total_educational_attainment_population_25_to_64_years_bachelors_degree_or_higher,labor_force_participation_rate_population_16_years_and_over,labor_force_participation_rate_population_16_years_and_over_age_16_to_19_years,labor_force_participation_rate_population_16_years_and_over_age_20_to_24_years,labor_force_participation_rate_population_16_years_and_over_age_25_to_29_years,labor_force_participation_rate_population_16_years_and_over_age_30_to_34_years,labor_force_participation_rate_population_16_years_and_over_age_35_to_44_years,labor_force_participation_rate_population_16_years_and_over_age_45_to_54_years,labor_force_participation_rate_population_16_years_and_over_age_55_to_59_years,labor_force_participation_rate_population_16_years_and_over_age_60_to_64_years,labor_force_participation_rate_population_16_years_and_over_age_65_to_74_years,labor_force_participation_rate_population_16_years_and_over_age_75_years_and_over,labor_force_participation_rate_population_16_years_and_over_race_and_hispanic_or_latino_origin_american_indian_and_alaska_native_alone,labor_force_participation_rate_population_16_years_and_over_race_and_hispanic_or_latino_origin_native_hawaiian_and_other_pacific_islander_alone,labor_force_participation_rate_population_16_years_and_over_race_and_hispanic_or_latino_origin_hispanic_or_latino_origin_of_any_race,labor_force_participation_rate_population_20_to_64_years,labor_force_participation_rate_population_20_to_64_years_poverty_status_in_the_past_12_months_below_poverty_level,labor_force_participation_rate_population_20_to_64_years_poverty_status_in_the_past_12_months_at_or_above_the_poverty_level,labor_force_participation_rate_population_20_to_64_years_disability_status_with_any_disability,labor_force_participation_rate_educational_attainment_population_25_to_64_years,labor_force_participation_rate_educational_attainment_population_25_to_64_years_less_than_high_school_graduate,labor_force_participation_rate_educational_attainment_population_25_to_64_years_high_school_graduate_includes_equivalency,labor_force_participation_rate_educational_attainment_population_25_to_64_years_some_college_or_associates_degree,labor_force_participation_rate_educational_attainment_population_25_to_64_years_bachelors_degree_or_higher,employment/population_ratio_population_16_years_and_over,employment/population_ratio_population_16_years_and_over_age_16_to_19_years,employment/population_ratio_popu

In [28]:
# Get column list
s2301_cols = dfs["s2301"].columns.tolist()

# Print them all
for col in s2301_cols:
    print(col)

geography
geographic_area_name
total_population_16_years_and_over
total_population_16_years_and_over_age_16_to_19_years
total_population_16_years_and_over_age_20_to_24_years
total_population_16_years_and_over_age_25_to_29_years
total_population_16_years_and_over_age_30_to_34_years
total_population_16_years_and_over_age_35_to_44_years
total_population_16_years_and_over_age_45_to_54_years
total_population_16_years_and_over_age_55_to_59_years
total_population_16_years_and_over_age_60_to_64_years
total_population_16_years_and_over_age_65_to_74_years
total_population_16_years_and_over_age_75_years_and_over
total_population_16_years_and_over_race_and_hispanic_or_latino_origin_american_indian_and_alaska_native_alone
total_population_16_years_and_over_race_and_hispanic_or_latino_origin_native_hawaiian_and_other_pacific_islander_alone
total_population_16_years_and_over_race_and_hispanic_or_latino_origin_hispanic_or_latino_origin_of_any_race
total_population_20_to_64_years
total_population_20_to

In [29]:
# Define output path
output_path = "/cleaned_s2301.csv"

# Save the cleaned DataFrame
dfs["s2301"].to_csv(output_path, index=False)
print(f"✅ Saved: {output_path}")

✅ Saved: /Users/marcomagnolo/Desktop/NEXT/pro_bono_projects/ahsas_school/DATA/clean_datasets/cleaned_s2301.csv


# 🧼 Cleaning Summary: `s2301` – Employment Status

**Source**: `ACSST5Y2021.S2301-Data.csv`  
**Location**: `/DATA/ACSST5Y2021.S2301_2025-06-06T064553/`  
**Output**: `cleaned_s2301.csv` in `/DATA/clean_datasets/`

---

## ✅ Cleaning Steps

1. **Column Filtering**
   - Kept:
     - `Geography`, `Geographic Area Name`
     - All `Estimate!!` columns related to:
       - Total population 16+ and by age group
       - Labor force participation, employment/population ratio, and unemployment rate
       - Breakdowns by poverty status, disability, and educational attainment
   - Removed:
     - All `Margin of Error!!` columns
     - All columns with deeply nested demographic splits (e.g., `!!SEX!!`, `!!RACE!!`, or `!!White alone!!Male!!Age!!...`)

2. **Column Name Standardization**
   - Converted to `snake_case`
   - Removed special characters (`!!`, commas, dashes, parentheses, etc.)

---

## 📊 Final Result

- **Rows**: 11 (one per target town)
- **Columns**: 94  
- **Key Themes**:
  - Labor force participation across age brackets
  - Employment and unemployment rates
  - Socioeconomic and education-related workforce gaps
  - Limited, strategic retention of race/ethnicity (e.g., Hispanic, AIAN, NHPI)

---

## 🧠 Notes

> `s2301` offers detailed insight into how employment varies by age, education, and socio-economic factors.  
> This cleaned version retains actionable data for outreach, program eligibility modeling, and workforce segmentation while avoiding overwhelming or redundant breakdowns.

✔️ Cleaned and ready for integration into the master dataset.

In [30]:
def clean_s2403(df):
    # Step 1: Keep only Geography and Estimate columns
    cols_to_keep = ['Geography', 'Geographic Area Name'] + [
        col for col in df.columns if col.startswith("Estimate!!")
    ]
    df = df[cols_to_keep].copy()

    # Step 2: Drop male/female breakdowns, earnings data, and sub-industry splits
    drop_keywords = [
        "!!Male!!", "!!Female!!",
        "!!Median earnings", "!!Mean earnings",
        "!!SEX!!",
        "!!Agriculture, forestry, fishing and hunting, and mining:!!",
        "!!Manufacturing:!!",  # subcategories within sectors
        "!!Construction:!!",
        "!!Retail trade:!!",
        "!!Finance and insurance:!!",
        "!!Educational services, and health care and social assistance:!!",
        "!!Public administration:!!"
    ]
    df = df[[col for col in df.columns if not any(kw in col for kw in drop_keywords)]]

    # Step 3: Standardize column names
    def clean_col(col):
        if col in ['Geography', 'Geographic Area Name']:
            return col.lower().replace(" ", "_")
        col = col.replace("Estimate!!", "")
        col = col.replace("!!", "_")
        col = col.replace(" ", "_")
        col = col.replace(",", "")
        col = col.replace("(", "")
        col = col.replace(")", "")
        col = col.replace("-", "_")
        col = col.replace(":", "")
        return col.lower()

    df.columns = [clean_col(c) for c in df.columns]

    return df

In [31]:
dfs["s2403"] = clean_s2403(dfs["s2403"])
print(f"✅ Cleaned s2403 shape: {dfs['s2403'].shape}")
dfs["s2403"].head()

✅ Cleaned s2403 shape: (11, 71)


,geography,geographic_area_name,total_civilian_employed_population_16_years_and_over,total_civilian_employed_population_16_years_and_over_agriculture_forestry_fishing_and_hunting_and_mining,total_civilian_employed_population_16_years_and_over_construction,total_civilian_employed_population_16_years_and_over_manufacturing,total_civilian_employed_population_16_years_and_over_wholesale_trade,total_civilian_employed_population_16_years_and_over_retail_trade,total_civilian_employed_population_16_years_and_over_transportation_and_warehousing_and_utilities,total_civilian_employed_population_16_years_and_over_transportation_and_warehousing_and_utilities_transportation_and_warehousing,total_civilian_employed_population_16_years_and_over_transportation_and_warehousing_and_utilities_utilities,total_civilian_employed_population_16_years_and_over_information,total_civilian_employed_population_16_years_and_over_finance_and_insurance_and_real_estate_and_rental_and_leasing,total_civilian_employed_population_16_years_and_over_finance_and_insurance_and_real_estate_and_rental_and_leasing_finance_and_insurance,total_civilian_employed_population_16_years_and_over_finance_and_insurance_and_real_estate_and_rental_and_leasing_real_estate_and_rental_and_leasing,total_civilian_employed_population_16_years_and_over_professional_scientific_and_management_and_administrative_and_waste_management_services,total_civilian_employed_population_16_years_and_over_professional_scientific_and_management_and_administrative_and_waste_management_services_professional_scientific_and_technical_services,total_civilian_employed_population_16_years_and_over_professional_scientific_and_management_and_administrative_and_waste_management_services_management_of_companies_and_enterprises,total_civilian_employed_population_16_years_and_over_professional_scientific_and_management_and_administrative_and_waste_management_services_administrative_and_support_and_waste_management_services,total_civilian_employed_population_16_years_and_over_educational_services_and_health_care_and_social_assistance,total_civilian_employed_population_16_years_and_over_arts_entertainment_and_recreation_and_accommodation_and_food_services,total_civilian_employed_population_16_years_and_over_arts_entertainment_and_recreation_and_accommodation_and_food_services_arts_entertainment_and_recreation,total_civilian_employed_population_16_years_and_over_arts_entertainment_and_recreation_and_accommodation_and_food_services_accommodation_and_food_services,total_civilian_employed_population_16_years_and_over_other_services_except_public_administration,total_civilian_employed_population_16_years_and_over_public_administration,percent_male_civilian_employed_population_16_years_and_over,percent_male_civilian_employed_population_16_years_and_over_agriculture_forestry_fishing_and_hunting_and_mining,percent_male_civilian_employed_population_16_years_and_over_construction,percent_male_civilian_employed_population_16_years_and_over_manufacturing,percent_male_civilian_employed_population_16_years_and_over_wholesale_trade,percent_male_civilian_employed_population_16_years_and_over_retail_trade,percent_male_civilian_employed_population_16_years_and_over_transportation_and_warehousing_and_utilities,percent_male_civilian_employed_population_16_years_and_over_transportation_and_warehousing_and_utilities_transportation_and_warehousing,percent_male_civilian_employed_population_16_years_and_over_transportation_and_warehousing_and_utilities_utilities,percent_male_civilian_employed_population_16_years_and_over_information,percent_male_civilian_employed_population_16_years_and_over_finance_and_insurance_and_real_estate_and_rental_and_leasing,percent_male_civilian_employed_population_16_years_and_over_finance_and_insurance_and_real_estate_and_rental_and_leasing_finance_and_insurance,percent_male_civilian_employed_population_16_years_and_over_finance_and_insurance_and_real_estate_and_rental_and_leasing_real_estate_and_rental_and_leasing,

In [32]:
def refined_clean_s2403(df):
    # Drop all percent-male/female columns and sub-industry detail
    drop_prefixes = [
        "percent_male_",
        "percent_female_",
        "total_civilian_employed_population_16_years_and_over_transportation_and_warehousing_and_utilities_",
        "total_civilian_employed_population_16_years_and_over_finance_and_insurance_and_real_estate_and_rental_and_leasing_",
        "total_civilian_employed_population_16_years_and_over_professional_scientific_and_management_and_administrative_and_waste_management_services_",
        "total_civilian_employed_population_16_years_and_over_arts_entertainment_and_recreation_and_accommodation_and_food_services_"
    ]

    # Apply filtering
    df = df[[col for col in df.columns if not any(col.startswith(prefix) for prefix in drop_prefixes)]]

    return df

In [33]:
# Apply the final trim to s2403
dfs["s2403"] = refined_clean_s2403(dfs["s2403"])
print(f"✅ Refined s2403 shape: {dfs['s2403'].shape}")
dfs["s2403"].head()

✅ Refined s2403 shape: (11, 16)


,geography,geographic_area_name,total_civilian_employed_population_16_years_and_over,total_civilian_employed_population_16_years_and_over_agriculture_forestry_fishing_and_hunting_and_mining,total_civilian_employed_population_16_years_and_over_construction,total_civilian_employed_population_16_years_and_over_manufacturing,total_civilian_employed_population_16_years_and_over_wholesale_trade,total_civilian_employed_population_16_years_and_over_retail_trade,total_civilian_employed_population_16_years_and_over_transportation_and_warehousing_and_utilities,total_civilian_employed_population_16_years_and_over_information,total_civilian_employed_population_16_years_and_over_finance_and_insurance_and_real_estate_and_rental_and_leasing,total_civilian_employed_population_16_years_and_over_professional_scientific_and_management_and_administrative_and_waste_management_services,total_civilian_employed_population_16_years_and_over_educational_services_and_health_care_and_social_assistance,total_civilian_employed_population_16_years_and_over_arts_entertainment_and_recreation_and_accommodation_and_food_services,total_civilian_employed_population_16_years_and_over_other_services_except_public_administration,total_civilian_employed_population_16_years_and_over_public_administration
0,0600000US0900302060,"Avon town, Hartford County, Connecticut",9069,75,205,1257,225,546,66,370,1575,1321,2241,631,283,274
1,0600000US0900305910,"Bloomfield town, Hartford County, Connecticut",10996,0,600,1019,95,614,734,155,1220,873,3808,517,749,612
2,0600000US0900322070,"East Granby town, Hartford County, Connecticut",2727,1,239,332,0,314,316,91,259,301,515,175,115,69
3,0600000US0900332640,"Granby town, Hartford County, Connecticut",5795,63,560,722,138,604,139,108,795,732,1127,364,211,232
4,0600000US0900337070,"Hartford town, Hartford County, Connecticut",51544,78,2557,3249,1204,7085,4763,599,3081,5411,13595,5049,2558,2315


In [34]:
# Get column list
s2403_cols = dfs["s2403"].columns.tolist()

# Print them all
for col in s2403_cols:
    print(col)

geography
geographic_area_name
total_civilian_employed_population_16_years_and_over
total_civilian_employed_population_16_years_and_over_agriculture_forestry_fishing_and_hunting_and_mining
total_civilian_employed_population_16_years_and_over_construction
total_civilian_employed_population_16_years_and_over_manufacturing
total_civilian_employed_population_16_years_and_over_wholesale_trade
total_civilian_employed_population_16_years_and_over_retail_trade
total_civilian_employed_population_16_years_and_over_transportation_and_warehousing_and_utilities
total_civilian_employed_population_16_years_and_over_information
total_civilian_employed_population_16_years_and_over_finance_and_insurance_and_real_estate_and_rental_and_leasing
total_civilian_employed_population_16_years_and_over_professional_scientific_and_management_and_administrative_and_waste_management_services
total_civilian_employed_population_16_years_and_over_educational_services_and_health_care_and_social_assistance
total_civilia

In [35]:
# Define output path
output_path = "/Users/marcomagnolo/Desktop/NEXT/pro_bono_projects/ahsas_school/DATA/clean_datasets/cleaned_s2403.csv"

# Save the cleaned DataFrame
dfs["s2403"].to_csv(output_path, index=False)
print(f"✅ Saved: {output_path}")

✅ Saved: /Users/marcomagnolo/Desktop/NEXT/pro_bono_projects/ahsas_school/DATA/clean_datasets/cleaned_s2403.csv


# 🧼 Cleaning Summary: `s2403` – Industry by Employment

**Source**: `ACSST5Y2021.S2403-Data.csv`  
**Location**: `/DATA/ACSST5Y2021.S2403_2025-06-06T064745/`  
**Output**: `cleaned_s2403.csv` in `/DATA/clean_datasets/`

---

## ✅ Cleaning Steps

1. **Base Cleaning**
   - Kept:
     - `Geography`, `Geographic Area Name`
     - All `Estimate!!` columns
   - Removed:
     - All `Margin of Error!!` columns
     - All male/female breakdowns
     - All median and mean earnings columns

2. **Refined Cleaning**
   - Removed all:
     - `percent_male_...` and `percent_female_...` columns
     - Sub-industry details under umbrella sectors (e.g., separate entries for `utilities`, `waste management`, etc.)

3. **Column Name Standardization**
   - Converted to `snake_case`
   - Removed special characters (`!!`, colons, parentheses, dashes, etc.)

---

## 📊 Final Result

- **Rows**: 11 (one per target town)
- **Columns**: 16
- **Key Themes**:
  - Total employment across major industry sectors, including:
    - Construction
    - Manufacturing
    - Retail
    - Education and Healthcare
    - Public Administration
    - Finance, Information, and more

---

## 🧠 Notes

> `s2403` offers a clean industry-level view of employment patterns in each community.  
> The refined version focuses on **sector size**, leaving out gender splits and micro-sector noise, making it well-suited for persona design, local economic profiling, and employment-related outreach.

✔️ Cleaned and ready for master dataset integration.

In [36]:
def clean_c15002(df):
    # 1. Keep only 'Geography', 'Geographic Area Name', and Estimate columns
    cols_to_keep = ['Geography', 'Geographic Area Name'] + [
        col for col in df.columns if col.startswith("Estimate!!")
    ]
    df = df[cols_to_keep].copy()

    # 2. Standardize column names
    def clean_col(col):
        if col in ['Geography', 'Geographic Area Name']:
            return col.lower().replace(" ", "_")
        col = col.replace("Estimate!!", "")
        col = col.replace("!!", "_")
        col = col.replace(" ", "_")
        col = col.replace(",", "")
        col = col.replace("(", "")
        col = col.replace(")", "")
        col = col.replace("-", "_")
        col = col.replace("'", "")
        return col.lower()

    df.columns = [clean_col(c) for c in df.columns]
    return df

# 🔁 Loop through and clean all c15002 tables
for key in [k for k in dfs.keys() if k.startswith("c15002")]:
    dfs[key] = clean_c15002(dfs[key])
    print(f"✅ Cleaned: {key} → {dfs[key].shape[0]} rows, {dfs[key].shape[1]} columns")

✅ Cleaned: c15002a → 11 rows, 13 columns
✅ Cleaned: c15002b → 11 rows, 13 columns
✅ Cleaned: c15002c → 11 rows, 13 columns
✅ Cleaned: c15002d → 11 rows, 13 columns
✅ Cleaned: c15002e → 11 rows, 13 columns
✅ Cleaned: c15002f → 11 rows, 13 columns
✅ Cleaned: c15002g → 11 rows, 13 columns
✅ Cleaned: c15002h → 11 rows, 13 columns
✅ Cleaned: c15002i → 11 rows, 13 columns


In [37]:
# Show the first 5 rows of the cleaned c15002a DataFrame
dfs["c15002a"].head()

,geography,geographic_area_name,total:,total:_male:,total:_male:_less_than_high_school_diploma,total:_male:_high_school_graduate_includes_equivalency,total:_male:_some_college_or_associates_degree,total:_male:_bachelors_degree_or_higher,total:_female:,total:_female:_less_than_high_school_diploma,total:_female:_high_school_graduate_includes_equivalency,total:_female:_some_college_or_associates_degree,total:_female:_bachelors_degree_or_higher
0,0600000US0900302060,"Avon town, Hartford County, Connecticut",11176,5373,141,394,808,4030,5803,225,651,1575,3352
1,0600000US0900305910,"Bloomfield town, Hartford County, Connecticut",6165,2879,117,520,667,1575,3286,76,477,904,1829
2,0600000US0900322070,"East Granby town, Hartford County, Connecticut",2870,1389,38,247,444,660,1481,115,162,415,789
3,0600000US0900332640,"Granby town, Hartford County, Connecticut",7578,3919,66,890,868,2095,3659,77,668,662,2252
4,0600000US0900337070,"Hartford town, Hartford County, Connecticut",20628,10043,2761,2779,1857,2646,10585,2754,2814,2351,2666


In [38]:
# 📁 Define output directory
output_dir = "/Users/marcomagnolo/Desktop/NEXT/pro_bono_projects/ahsas_school/DATA/clean_datasets"

# 💾 Save each cleaned c15002 DataFrame
for key in [k for k in dfs.keys() if k.startswith("c15002")]:
    output_path = f"{output_dir}/cleaned_{key}.csv"
    dfs[key].to_csv(output_path, index=False)
    print(f"✅ Saved: {output_path}")

✅ Saved: /Users/marcomagnolo/Desktop/NEXT/pro_bono_projects/ahsas_school/DATA/clean_datasets/cleaned_c15002a.csv
✅ Saved: /Users/marcomagnolo/Desktop/NEXT/pro_bono_projects/ahsas_school/DATA/clean_datasets/cleaned_c15002b.csv
✅ Saved: /Users/marcomagnolo/Desktop/NEXT/pro_bono_projects/ahsas_school/DATA/clean_datasets/cleaned_c15002c.csv
✅ Saved: /Users/marcomagnolo/Desktop/NEXT/pro_bono_projects/ahsas_school/DATA/clean_datasets/cleaned_c15002d.csv
✅ Saved: /Users/marcomagnolo/Desktop/NEXT/pro_bono_projects/ahsas_school/DATA/clean_datasets/cleaned_c15002e.csv
✅ Saved: /Users/marcomagnolo/Desktop/NEXT/pro_bono_projects/ahsas_school/DATA/clean_datasets/cleaned_c15002f.csv
✅ Saved: /Users/marcomagnolo/Desktop/NEXT/pro_bono_projects/ahsas_school/DATA/clean_datasets/cleaned_c15002g.csv
✅ Saved: /Users/marcomagnolo/Desktop/NEXT/pro_bono_projects/ahsas_school/DATA/clean_datasets/cleaned_c15002h.csv
✅ Saved: /Users/marcomagnolo/Desktop/NEXT/pro_bono_projects/ahsas_school/DATA/clean_datasets/cle

## 📊 Cleaning Summary – Race × Sex × Education Tables (`c15002a` to `c15002i`)

### ✅ What These Tables Contain
The `c15002X` series provides detailed cross-tabulations of:
- **Sex (male/female)**
- **Educational attainment**
- For **one specific racial/ethnic group per table**

These are essential for fine-grained persona analysis across race and gender lines.

### 🧹 Cleaning Steps
For each dataset:
1. **Dropped Margin of Error columns**
2. **Kept only `Estimate!!` values** and identifiers (`Geography`, `Geographic Area Name`)
3. **Standardized column names** to `snake_case` for consistency

Example output columns:
- `male_total`
- `male_less_than_high_school_diploma`
- `female_bachelors_degree_or_higher`

### 💾 Saved Files
Each cleaned DataFrame was saved as a separate CSV

# 📊 Dataset Overview – Ahsas School Community Data (Cleaned CSVs)

## Demographic & Identity

- **`cleaned_b02001.csv`**  
  ➤ Race breakdown (e.g., White, Black, Asian, Native American)  
  ➤ Total population by race

- **`cleaned_b03002.csv`**  
  ➤ Hispanic/Latino origin by race  
  ➤ Distinguishes between Hispanic and Non-Hispanic population by race

---

## Educational Attainment

- **`cleaned_s1501.csv`**  
  ➤ Education levels by age group, sex, and race  
  ➤ Includes poverty rates and median earnings by education

- **`cleaned_c15002[a-i].csv`**  
  ➤ One file per race group (White, Black, Asian, etc.)  
  ➤ Cross-tab of **sex × education**  
  ➤ Supports precise segmentation (e.g., "Latino women with high school diploma")

---

## Household & Family Structure

- **`cleaned_s0101.csv`**  
  ➤ Total population by age and sex  
  ➤ Foundation for population pyramids and age-targeted strategies

- **`cleaned_s1101.csv`**  
  ➤ Family vs. non-family households  
  ➤ Presence of children, seniors, and people living alone

---

## Income & Socioeconomics

- **`cleaned_s1901.csv`**  
  ➤ Household, family, and nonfamily income brackets  
  ➤ Median and mean income by household type

---

## Employment & Education/Labor Linkages

- **`cleaned_s2301.csv`**  
  ➤ Labor force participation, employment rate, unemployment rate  
  ➤ Breakdown by age, education, disability, and poverty status

- **`cleaned_s2403.csv`**  
  ➤ Employment by industry sector (e.g., manufacturing, health care)  
  ➤ Total workers by sector (gender removed for simplicity)

In [39]:
# Let's prepare our single dataset for merging
# we clean, normalize and tag source cols (so to know what's their root df
# once we merge them into the master_df)

def prepare_for_merge(df, source_prefix):
    """
    Cleans and tags a single cleaned DataFrame prior to merging.

    Args:
        df (pd.DataFrame): The loaded cleaned dataset.
        source_prefix (str): A short tag to prefix column names (e.g. 's1501', 'b02001').

    Returns:
        pd.DataFrame: The cleaned and tagged DataFrame.
    """

    # 1. Standardize missing values
    df.replace(to_replace=["(X)", "-", "null", "", "N", "n.a."], value=np.nan, inplace=True)

    # 2. Attempt to convert all non-ID columns to float
    for col in df.columns:
        if col not in ['geography', 'geographic_area_name']:
            df[col] = pd.to_numeric(df[col], errors='ignore')

    # 3. Prefix all columns (except join keys) with dataset tag
    df = df.rename(columns=lambda c: f"{source_prefix}_{c}" if c not in ['geography', 'geographic_area_name'] else c)

    return df

In [40]:
# 📥 Load from cleaned CSVs
clean_dir = "/Users/marcomagnolo/Desktop/NEXT/pro_bono_projects/ahsas_school/DATA/clean_datasets"
keys = ["s0101", "s1101", "s1501", "s1901", "s2301", "s2403", "b02001", "b03002"]
dfs = {}

for key in keys:
    dfs[key] = pd.read_csv(f"{clean_dir}/cleaned_{key}.csv")
    dfs[key] = prepare_for_merge(dfs[key], key)

# 🔁 Loop through and clean c15002a–i
for key in ["c15002a","c15002b","c15002c","c15002d","c15002e","c15002f","c15002g","c15002h","c15002i"]:
    dfs[key] = pd.read_csv(f"{clean_dir}/cleaned_{key}.csv")
    dfs[key] = prepare_for_merge(dfs[key], key)

In [41]:
for name, df in dfs.items():
    print(f"🧾 Dataset: {name}")
    print(f"➡️  Number of columns: {df.shape[1]}")
    print("📑 Column names:")
    print(df.columns.tolist())
    print("-" * 80)

🧾 Dataset: s0101
➡️  Number of columns: 98
📑 Column names:
['geography', 's0101_town', 's0101_total_total_population', 's0101_total_total_population_age_under_5_years', 's0101_total_total_population_age_5_to_9_years', 's0101_total_total_population_age_10_to_14_years', 's0101_total_total_population_age_15_to_19_years', 's0101_total_total_population_age_20_to_24_years', 's0101_total_total_population_age_25_to_29_years', 's0101_total_total_population_age_30_to_34_years', 's0101_total_total_population_age_35_to_39_years', 's0101_total_total_population_age_40_to_44_years', 's0101_total_total_population_age_45_to_49_years', 's0101_total_total_population_age_50_to_54_years', 's0101_total_total_population_age_55_to_59_years', 's0101_total_total_population_age_60_to_64_years', 's0101_total_total_population_age_65_to_69_years', 's0101_total_total_population_age_70_to_74_years', 's0101_total_total_population_age_75_to_79_years', 's0101_total_total_population_age_80_to_84_years', 's0101_total_tota

In [42]:
# 🛠️ Fix the key column in s0101
dfs["s0101"].rename(columns={"s0101_town": "geographic_area_name"}, inplace=True)


In [43]:
# 📦 Overwrite all cleaned CSVs with merged-ready versions
for key, df in dfs.items():
    output_path = f"{clean_dir}/cleaned_{key}.csv"
    df.to_csv(output_path, index=False)
    print(f"✅ Overwritten: {output_path}")

✅ Overwritten: /Users/marcomagnolo/Desktop/NEXT/pro_bono_projects/ahsas_school/DATA/clean_datasets/cleaned_s0101.csv
✅ Overwritten: /Users/marcomagnolo/Desktop/NEXT/pro_bono_projects/ahsas_school/DATA/clean_datasets/cleaned_s1101.csv
✅ Overwritten: /Users/marcomagnolo/Desktop/NEXT/pro_bono_projects/ahsas_school/DATA/clean_datasets/cleaned_s1501.csv
✅ Overwritten: /Users/marcomagnolo/Desktop/NEXT/pro_bono_projects/ahsas_school/DATA/clean_datasets/cleaned_s1901.csv
✅ Overwritten: /Users/marcomagnolo/Desktop/NEXT/pro_bono_projects/ahsas_school/DATA/clean_datasets/cleaned_s2301.csv
✅ Overwritten: /Users/marcomagnolo/Desktop/NEXT/pro_bono_projects/ahsas_school/DATA/clean_datasets/cleaned_s2403.csv
✅ Overwritten: /Users/marcomagnolo/Desktop/NEXT/pro_bono_projects/ahsas_school/DATA/clean_datasets/cleaned_b02001.csv
✅ Overwritten: /Users/marcomagnolo/Desktop/NEXT/pro_bono_projects/ahsas_school/DATA/clean_datasets/cleaned_b03002.csv
✅ Overwritten: /Users/marcomagnolo/Desktop/NEXT/pro_bono_proje

In [44]:
# LET'S CREATE  A MASTER_DF with all the single df merged on the keys
# 🔁 Merge all datasets on shared keys
from functools import reduce

# List of DataFrames to merge
dfs_to_merge = list(dfs.values())

# Merge on 'geography' and 'geographic_area_name'
master_df = reduce(lambda left, right: pd.merge(
    left, right, on=["geography", "geographic_area_name"], how="outer"), dfs_to_merge)

# 🧼 Optional: Final clean-up of column names
master_df.columns = (
    master_df.columns
    .str.replace(":", "", regex=False)
    .str.replace("$", "", regex=False)
    .str.replace(r"_+", "_", regex=True)
    .str.strip("_")
)

# 💾 Save master dataset
master_path = f"{clean_dir}/master_df.csv"
master_df.to_csv(master_path, index=False)
print(f"✅ Master dataset saved to: {master_path}")
print(f"🧾 Final shape: {master_df.shape}")

✅ Master dataset saved to: /Users/marcomagnolo/Desktop/NEXT/pro_bono_projects/ahsas_school/DATA/clean_datasets/master_df.csv
🧾 Final shape: (11, 462)


## 🧼 Final Cleaning & Merging Datasets

### 🗂 Step 1: Load Cleaned Individual Datasets
We load the previously saved and lightly cleaned CSVs for:

- `s0101`: Age and sex distribution
- `s1101`: Household and family structure
- `s1501`: Educational attainment (detailed)
- `s1901`: Household and family income
- `s2301`: Labor force and employment status
- `s2403`: Industry-based employment
- `b02001`: Race
- `b03002`: Race by Hispanic/Latino ethnicity
- `c15002a–i`: Educational attainment by race and sex

### 🏷 Step 2: Normalize and Tag with `prepare_for_merge()`
Each dataset is passed through a standardization function:
- Replaces placeholders like `-`, `(X)`, `null` with `NaN`
- Converts numeric-looking columns to float where possible
- Prefixes all columns (except `geography` and `geographic_area_name`) with a dataset-specific tag  
  _Example: `s1501_median_earnings_25_plus`_

This ensures traceability and uniform formatting ahead of merging.

### 🔄 Step 3: Overwrite CSVs with Cleaned Versions
We overwrite the previously saved cleaned CSVs to reflect the standardized and tagged structure post `prepare_for_merge()`.

### 🔀 Step 4: Merge into `master_df`
All datasets are merged using an `outer join` on:

- `geography`
- `geographic_area_name`

This ensures full preservation of all geographic entities across datasets.

### 🧹 Step 5: Final Column Name Cleanup
In `master_df`, column names are normalized by removing:
- Colons (`:`)
- Dollar signs (`$`)
- Repeated or trailing underscores (`_`)

💾 Step 6: Save Master Dataset
The final merged dataset is saved

✅ Final shape: 11 rows × 462 columns
Ready for exploration and segmentation.

In [45]:
master_df.describe(include='all')

,geography,geographic_area_name,s0101_total_total_population,s0101_total_total_population_age_under_5_years,s0101_total_total_population_age_5_to_9_years,s0101_total_total_population_age_10_to_14_years,s0101_total_total_population_age_15_to_19_years,s0101_total_total_population_age_20_to_24_years,s0101_total_total_population_age_25_to_29_years,s0101_total_total_population_age_30_to_34_years,s0101_total_total_population_age_35_to_39_years,s0101_total_total_population_age_40_to_44_years,s0101_total_total_population_age_45_to_49_years,s0101_total_total_population_age_50_to_54_years,s0101_total_total_population_age_55_to_59_years,s0101_total_total_population_age_60_to_64_years,s0101_total_total_population_age_65_to_69_years,s0101_total_total_population_age_70_to_74_years,s0101_total_total_population_age_75_to_79_years,s0101_total_total_population_age_80_to_84_years,s0101_total_total_population_age_85_years_and_over,s0101_total_total_population_selected_age_categories_5_to_14_years,s0101_total_total_population_selected_age_categories_15_to_17_years,s0101_total_total_population_selected_age_categories_under_18_years,s0101_total_total_population_selected_age_categories_18_to_24_years,s0101_total_total_population_selected_age_categories_15_to_44_years,s0101_total_total_population_selected_age_categories_16_years_and_over,s0101_total_total_population_selected_age_categories_18_years_and_over,s0101_total_total_population_selected_age_categories_21_years_and_over,s0101_total_total_population_selected_age_categories_60_years_and_over,s0101_total_total_population_selected_age_categories_62_years_and_over,s0101_total_total_population_selected_age_categories_65_years_and_over,s0101_total_total_population_selected_age_categories_75_years_and_over,s0101_total_total_population_summary_indicators_median_age_years,s0101_male_total_population,s0101_male_total_population_age_under_5_years,s0101_male_total_population_age_5_to_9_years,s0101_male_total_population_age_10_to_14_years,s0101_male_total_population_age_15_to_19_years,s0101_male_total_population_age_20_to_24_years,s0101_male_total_population_age_25_to_29_years,s0101_male_total_population_age_30_to_34_years,s0101_male_total_population_age_35_to_39_years,s0101_male_total_population_age_40_to_44_years,s0101_male_total_population_age_45_to_49_years,s0101_male_total_population_age_50_to_54_years,s0101_male_total_population_age_55_to_59_years,s0101_male_total_population_age_60_to_64_years,s0101_male_total_population_age_65_to_69_years,s0101_male_total_population_age_70_to_74_years,s0101_male_total_population_age_75_to_79_years,s0101_male_total_population_age_80_to_84_years,s0101_male_total_population_age_85_years_and_over,s0101_male_total_population_selected_age_categories_5_to_14_years,s0101_male_total_population_selected_age_categories_15_to_17_years,s0101_male_total_population_selected_age_categories_under_18_years,s0101_male_total_population_selected_age_categories_18_to_24_years,s0101_male_total_population_selected_age_categories_15_to_44_years,s0101_male_total_population_selected_age_categories_16_years_and_over,s0101_male_total_population_selected_age_categories_18_years_and_over,s0101_male_total_population_selected_age_categories_21_years_and_over,s0101_male_total_population_selected_age_categories_60_years_and_over,s0101_male_total_population_selected_age_categories_62_years_and_over,s0101_male_total_population_selected_age_categories_65_years_and_over,s0101_male_total_population_selected_age_categories_75_years_and_over,s0101_male_total_population_summary_indicators_median_age_years,s0101_female_total_population,s0101_female_total_population_age_under_5_years,s0101_female_total_population_age_5_to_9_years,s0101_female_total_population_age_10_to_14_years,s0101_female_total_population_age_15_to_19_years,s0101_female_total_population_age_20_to_24_years,s0101_female_total_population_age_25_to_29_years,s0101_female_total_population_age_30_to_34_years,s0101_female_total_population_age_35_to_39

In [50]:
import pandas as pd

# Step 1: Load the original dataset
input_path = "TAA/master_df.csv"
df = pd.read_csv(input_path)

# Step 2: Clean the town_name from 'geographic_area_name' column
df['town_name'] = (
    df['geographic_area_name']
    .str.replace(' town', '', case=False, regex=False)
    .str.replace('City of ', '', case=False, regex=False)
    .str.replace(', Hartford County, Connecticut', '', case=False, regex=False)
    .str.strip()
)

# Step 3: Add a column for Tableau geographic recognition
df['state'] = 'Connecticut'

# Step 4: Export the cleaned dataset
output_path = "TAA/master_df_tableau_ready.csv"
df.to_csv(output_path, index=False)

print("✅ Tableau-ready dataset saved to:")
print(output_path)

✅ Tableau-ready dataset saved to:
/Users/marcomagnolo/Desktop/NEXT/pro_bono_projects/ahsas_school/DATA/clean_datasets/master_df_tableau_ready.csv


In [51]:
df['town_name'].unique()

array(['Avon', 'Bloomfield', 'East Granby', 'Granby', 'Hartford',
       'Simsbury', 'South Windsor', 'Suffield', 'West Hartford',
       'Windsor', 'Windsor Locks'], dtype=object)

In [49]:
# Identify missing mappings
unmapped_geoids = df[df['town_name'].isna()][geo_col].unique()
print("❗️ Missing GEOIDs:")
print(unmapped_geoids)


❗️ Missing GEOIDs:
['0600000US0900368940' '0600000US0900371390' '0600000US0900374540'
 '0600000US0900382590' '0600000US0900387000' '0600000US0900387070']


In [52]:
import pandas as pd

# --- File paths ---
geo_path = "/CT_DCF_Town_Boundaries_7896655440039092805.csv"
data_path = "/master_df_tableau_ready.csv"
output_path = "/merged_town_boundaries_demographics.csv"

# --- Load files ---
df_geo = pd.read_csv(geo_path)
df_data = pd.read_csv(data_path)

# --- Merge on town name ---
df_merged = pd.merge(
    df_geo,
    df_data,
    left_on='town',
    right_on='town_name',
    how='inner'
)

# --- Save merged output ---
df_merged.to_csv(output_path, index=False)
print("✅ Merged file saved to:", output_path)

✅ Merged file saved to: /Users/marcomagnolo/Desktop/NEXT/pro_bono_projects/ahsas_school/DATA/clean_datasets/merged_town_boundaries_demographics.csv


In [53]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Columns: 475 entries, OBJECTID to state
dtypes: float64(129), int64(336), object(10)
memory usage: 40.9+ KB


In [55]:
import geopandas as gpd
import pandas as pd

# Load the shapefile
gdf = gpd.read_file("/CT_DCF_Town_Boundaries.shp")

# Project to WGS84 (lat/lon)
gdf = gdf.to_crs(epsg=4326)

# Extract polygon points for each town
rows = []
for idx, row in gdf.iterrows():
    town = row['town']
    geometry = row['geometry']
    if geometry.geom_type == 'Polygon':
        polygons = [geometry]
    elif geometry.geom_type == 'MultiPolygon':
        polygons = list(geometry)
    else:
        continue

    for poly_id, poly in enumerate(polygons):
        for coord in poly.exterior.coords:
            rows.append({
                'town': town,
                'segment_id': f"{town}_{poly_id}",
                'lon': coord[0],
                'lat': coord[1]
            })

# Convert to DataFrame
df_map = pd.DataFrame(rows)

# Save to CSV
output_path = "/ct_town_polygons_for_tableau.csv"
df_map.to_csv(output_path, index=False)
print("✅ CSV for Tableau Public saved to:", output_path)

✅ CSV for Tableau Public saved to: /Users/marcomagnolo/Desktop/NEXT/pro_bono_projects/ahsas_school/DATA/clean_datasets/ct_town_polygons_for_tableau.csv
